---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    state = ''
    region = ''
    towns = []
    for line in open("university_towns.txt"):
        if "[edit]" in line:
            state = line.split("[")[0]
        else:
            region = line.split(" (")[0].strip()
            towns.append([state, region])
    # convert the list to a datafram
    towns = pd.DataFrame(towns, columns=["State", "RegionName"])
    return towns
get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    # read-in the gdp data from the excel sheet, only use chained value dollars
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6],
                   names=['Quarter','GDP'])
    # only use the quarters data from 2000 onward
    gdp = gdp[gdp['Quarter'].str.startswith('2')]

    for i in range(1,len(gdp)-1):
        if gdp['GDP'].iloc[i] < gdp['GDP'].iloc[i-1] and gdp['GDP'].iloc[i] > gdp['GDP'].iloc[i+1]:
            return gdp['Quarter'].iloc[i]

get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    # read-in the gdp data from the excel sheet, only use chained value dollars
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6],
                   names=['Quarter','GDP'])
    # only use the quarters data from 2000 onward
    gdp = gdp[gdp['Quarter'].str.startswith('2')]
    
    # add a column for the year and the quarter number
    gdp["Year"] = gdp["Quarter"].str.split("q").str[0]
    gdp["Quarter#"] = gdp["Quarter"].str.split("q").str[1]
    
    # get the recession start year and quarter number
    startYear, startQuarter = get_recession_start().split("q")
    
    recession_start = gdp[(gdp["Year"] == startYear) & (gdp["Quarter#"] == startQuarter)].index[0]
    gdp = gdp.loc[recession_start:]
    
    # get the gdp numbers after the start of the recession
    for i in range(1,len(gdp)-1):
        if gdp['GDP'].iloc[i] > gdp['GDP'].iloc[i-1] and gdp['GDP'].iloc[i] < gdp['GDP'].iloc[i+1]:
            recession_end = gdp['Quarter'].iloc[i+1]
            return recession_end

get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    # read-in the gdp data from the excel sheet, only use chained value dollars
    gdp = pd.read_excel('gdplev.xls', skiprows=7, usecols=[4,6],
                   names=['Quarter','GDP'])
    # only use the quarters data from 2000 onward
    gdp = gdp[gdp['Quarter'].str.startswith('2')]
    
    # add a column for the year and the quarter number
    gdp["Year"] = gdp["Quarter"].str.split("q").str[0]
    gdp["Quarter#"] = gdp["Quarter"].str.split("q").str[1]
    
    # get the recession start year and quarter number
    startYear, startQuarter = get_recession_start().split("q")
    endYear, endQuarter = get_recession_end().split("q")
    
    recession_start = gdp[(gdp["Year"] == startYear) & (gdp["Quarter#"] == startQuarter)].index[0]
    recession_end = gdp[(gdp["Year"] == endYear) & (gdp["Quarter#"] == endQuarter)].index[0]
    gdp = gdp.loc[recession_start:recession_end]
    
    # return the min value during the recession
    return gdp["Quarter"].loc[gdp["GDP"].idxmin()]

get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # read-in the home sales from Zillow into a data frame
    home_sales = pd.read_csv('City_Zhvi_AllHomes.csv')
    home_sales["State"] = home_sales["State"].replace(states)
    
    # set the index
    home_sales = home_sales.set_index(["State", "RegionName"])
    
    # get only the columns for years 2000 onward
    the2000s = home_sales.columns[home_sales.columns.str.contains("^2")]
    home_sales = home_sales[the2000s]
    
    # break the years into their associated quarters
    quarters = [the2000s[i:i+3] for i in range(0, len(the2000s), 3)]
    
    # loop through the quarters and get their mean values
    for quarter in quarters:
        year = quarter[0].split('-')[0]
        q = int(quarter[0].split('-')[-1])//3 + 1
        col = "{}q{}".format(year, q)
        home_sales[col] = home_sales[quarter].mean(axis=1)
        home_sales.drop(quarter, axis=1, inplace=True)
    
    return home_sales

convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''

    # get all the years pertinent to the recession
    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()
    recession_end = get_recession_end()
    
    # price_ratio = quarter_before_recession/recession_bottom
    home_sales = convert_housing_data_to_quarters()
    # get the columns in the recession
    startIdx = home_sales.columns.get_loc(recession_start)
    bottomIdx = home_sales.columns.get_loc(recession_bottom)
    endIdx = home_sales.columns.get_loc(recession_end)
    # be sure to get the quarter before the recession starts
    # per the definition of the price ratio
    recession_columns = home_sales.columns[startIdx-1:bottomIdx+1]
    home_sales = home_sales[recession_columns]
    # reset the recession_start variable
    recession_start = home_sales.columns[0]
    # calculate the price ratio
    home_sales["Ratio"] = home_sales[recession_start].div(home_sales[recession_bottom])
    
    # get the list of university towns
    utowns = get_list_of_university_towns().set_index(["State", "RegionName"])
    
    home_sales_withU = home_sales.merge(utowns, how="inner", left_index=True, right_index=True)
    home_sales_withoutU_idx = home_sales.index.difference(home_sales_withU.index)
    home_sales_withoutU = home_sales.loc[home_sales_withoutU_idx]

    # run the ttest
    test = ttest_ind(home_sales_withU["Ratio"], home_sales_withoutU["Ratio"], nan_policy='omit')
    #print(test)
    tstat = test[0]
    p = test[1]
    if test[1] < 0.01:
        different = True
    else:
        different = False
        
    if tstat > 0:
        better = 'non-university town'
    else:
        better = 'university town'
    return (different, p, better)

run_ttest()